In [1]:
import os
import time
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, CuDNNLSTM, Embedding, Dropout, Activation, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.optimizers import Adam
from keras.models import Model
from keras import backend as K
from keras.engine.topology import Layer
from keras import initializers, regularizers, constraints, optimizers, layers

Using TensorFlow backend.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['embeddings', 'train.csv', 'sample_submission.csv', 'test.csv']


In [3]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = 100000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 100 # max number of words in a question to use

**Load packages and data**

In [4]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

Train shape :  (1306122, 3)
Test shape :  (375806, 2)


**Load embeddings**

In [5]:
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=2018)


## fill up the missing values
train_X = train_df["question_text"].fillna("_##_").values
val_X = val_df["question_text"].fillna("_##_").values
test_X = test_df["question_text"].fillna("_##_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X))
train_X = tokenizer.texts_to_sequences(train_X)
val_X = tokenizer.texts_to_sequences(val_X)
test_X = tokenizer.texts_to_sequences(test_X)

## Pad the sentences 
train_X = pad_sequences(train_X, maxlen=maxlen)
val_X = pad_sequences(val_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

## Get the target values
train_y = train_df['target'].values
val_y = val_df['target'].values
#shuffling the data
np.random.seed(2018)
trn_idx = np.random.permutation(len(train_X))
val_idx = np.random.permutation(len(val_X))

train_X = train_X[trn_idx]
val_X = val_X[val_idx]
train_y = train_y[trn_idx]
val_y = val_y[val_idx]

In [6]:
EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """


In [7]:
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Concatenate, Dropout, SpatialDropout1D
from keras.layers import Input, Embedding, Dense, Conv2D, MaxPool2D, concatenate

inp = Input(shape=(maxlen,))
x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
x = Bidirectional(CuDNNLSTM(128, return_sequences=True))(x)
x = Bidirectional(CuDNNLSTM(64,return_sequences=True))(x)

avg_pool = GlobalAveragePooling1D()(x)
max_pool = GlobalMaxPooling1D()(x)
conc = concatenate([avg_pool, max_pool])
conc = Dense(64, activation="relu")(conc)
conc = Dropout(0.1)(conc)
outp = Dense(1, activation="sigmoid")(conc)
model = Model(inputs=inp, outputs=outp)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
return model

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


SyntaxError: 'return' outside function (<ipython-input-7-2ee1e87b620b>, line 19)

In [8]:
model.fit(train_X, train_y, batch_size=512, epochs=4, validation_data=(val_X, val_y))

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1175509 samples, validate on 130613 samples
Epoch 1/4
1175509/1175509 [==============================] - 168s 143us/step - loss: 0.1171 - acc: 0.9542 - val_loss: 0.1045 - val_acc: 0.9581
Epoch 2/4
1175509/1175509 [==============================] - 164s 140us/step - loss: 0.1017 - acc: 0.9598 - val_loss: 0.0990 - val_acc: 0.9599
Epoch 3/4
 443392/1175509 [==========>...................] - ETA: 1:37 - loss: 0.0946 - acc: 0.9623

In [9]:
pred_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_val_y>thresh).astype(int))))


130613/130613 [==============================] - 6s 48us/step
F1 score at threshold 0.1 is 0.6076055160885917
F1 score at threshold 0.11 is 0.615923757425531
F1 score at threshold 0.12 is 0.6236484129752354
F1 score at threshold 0.13 is 0.6305313666251999
F1 score at threshold 0.14 is 0.6381305369734226
F1 score at threshold 0.15 is 0.6441768544903024
F1 score at threshold 0.16 is 0.6496268656716419
F1 score at threshold 0.17 is 0.6546384429606479
F1 score at threshold 0.18 is 0.6596848578016911
F1 score at threshold 0.19 is 0.6639420036004476
F1 score at threshold 0.2 is 0.6671919629666108
F1 score at threshold 0.21 is 0.6706204834288562
F1 score at threshold 0.22 is 0.6725779636253715
F1 score at threshold 0.23 is 0.674715981456009
F1 score at threshold 0.24 is 0.677931176591799
F1 score at threshold 0.25 is 0.679363097096472
F1 score at threshold 0.26 is 0.6814433531172857
F1 score at threshold 0.27 is 0.681948424068768
F1 score at threshold 0.28 is 0.683745109598585
F1 score at thr

In [10]:
pred_test_y = model.predict([test_X], batch_size=1024, verbose=1)

375806/375806 [==============================] - 17s 44us/step


In [11]:

thresholds = []
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    res = metrics.f1_score(val_y, (pred_val_y > thresh).astype(int))
    thresholds.append([thresh, res])
    print("F1 score at threshold {0} is {1}".format(thresh, res))
    
thresholds.sort(key=lambda x: x[1], reverse=True)
best_thresh = thresholds[0][0]
print("Best threshold: ", best_thresh)

F1 score at threshold 0.1 is 0.6076055160885917
F1 score at threshold 0.11 is 0.615923757425531
F1 score at threshold 0.12 is 0.6236484129752354
F1 score at threshold 0.13 is 0.6305313666251999
F1 score at threshold 0.14 is 0.6381305369734226
F1 score at threshold 0.15 is 0.6441768544903024
F1 score at threshold 0.16 is 0.6496268656716419
F1 score at threshold 0.17 is 0.6546384429606479
F1 score at threshold 0.18 is 0.6596848578016911
F1 score at threshold 0.19 is 0.6639420036004476
F1 score at threshold 0.2 is 0.6671919629666108
F1 score at threshold 0.21 is 0.6706204834288562
F1 score at threshold 0.22 is 0.6725779636253715
F1 score at threshold 0.23 is 0.674715981456009
F1 score at threshold 0.24 is 0.677931176591799
F1 score at threshold 0.25 is 0.679363097096472
F1 score at threshold 0.26 is 0.6814433531172857
F1 score at threshold 0.27 is 0.681948424068768
F1 score at threshold 0.28 is 0.683745109598585
F1 score at threshold 0.29 is 0.6849448410123297
F1 score at threshold 0.3 is

In [12]:
pred_test_y = (pred_test_y > best_thresh).astype(int)
out_df = pd.DataFrame({"qid":test_df["qid"].values})
out_df['prediction'] = pred_test_y
out_df.to_csv("submission.csv", index=False)